# 개요

- workflow
  - part1
    - 1. 사이트 접속 -> 응답 도착 (html : 웹문서)
      - 접속(요청)
      - 응답 : 웹문서를 긁어온다
  - part2
    - html -> 파싱 -> DOM(document object model)을 생성 (메모리에 상주)
      - DOM Tree를 구성한다
      - 파싱을 위해서는 파서 도구가 필요함
        - **html5lib**, html.parser, ...
        - html5lib : 상대적으로 느리나, HTML문서가 거대해도 누락없이 처리함
      - 라이브러리 : **bs4**

    - DOM을 통해서 탐색, 데이터 추출
      - 탐색(html에서 특정 대상을 특정한다, 찾는다)
        - 라이브러리 : bs4
        - 도구 : **css selector** | xpath | ...
      - 추출
        - 라이브러리 : bs4

- 결론
  - html -> bs4 -> 핋요한 정보 추출

## BS4

https://www.crummy.com/software/BeautifulSoup/bs4/doc.ko/

# part1 : 웹스크래핑하여 DOM 구성

- 타겟 사이트
  - 네이버 환율 정보 제공 사이트
  - https://finance.naver.com/marketindex/exchangeList.naver
  - 프레임 소스보기를 통해서 주소 획득
    - 페이지 안에 페이지 구성 => 프레임
  
- 특징
  - 어떤 은행도 무료로 정보를 제공하지 않는다(?)
    - 하나은행(외환은행 인수), 신한은행
    - 1일간 420회(?) 갱신 (고시회차)

- 목적
  - 통화명, 환율 관련 분석을 한다면?
  - 자동화
    - 08시 ~ 18시까지 1분 간격으로 진행된다(?)

In [38]:
# 1. 필요한 모듈 가져오기
import urllib.request as req

# 2. 타겟 사이트 설정
TARGET_SITE = 'https://finance.naver.com/marketindex/exchangeList.naver'

# 3. 요청(GET방식)
res = req.urlopen(TARGET_SITE)

In [39]:
# 4. 응답 결과중 HTML을 DOM Tree로 띄운다
from bs4 import BeautifulSoup # 코랩에서 기본 지원

# 5. 파싱 -> Dom Tree 구상
soup = BeautifulSoup(res,'html5lib') # 파서를 이용하여 파싱
soup

<html lang="ko"><head>
<title>네이버페이 증권</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="text/css" http-equiv="Content-Style-Type"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20250326145749/css/finance.css" rel="stylesheet" type="text/css"/>

<script src="https://ssl.pstatic.net/imgstock/static.pc/20250326145749/js/info/jindo.min.ns.1.5.3.euckr.js" type="text/javascript"></script>
<script src="https://ssl.pstatic.net/imgstock/static.pc/20250326145749/js/ntm.js" type="text/javascript"></script>
<script src="https://ssl.pstatic.net/imgstock/static.pc/20250326145749/js/nLog.js" type="text/javascript"></script>
</head>
<body>
<div class="tbl_area">
	<table border="1" class="tbl_exchange" summary="환전 고시 환율 리스트">
	<caption>환전 고시 환율</caption>
	<colgroup>
		<col width="162"/>
		<col width="92"/>
		<col width="92"/>
		<col width="92"/>
		<col width="93"/>
		<col width="92"/>

# part2 : DOM 탐색 -> 데이터 추출 -> 전처리 -> 적제

- 선행 기술
  - HTML내에서 특정 요소를 특정하는 기술
    - css selector or xpath or ....
  -css selector
    - 데이터를 감싸고 있는 요소(elements)를 특정하여 데이터를 추출함
      -용어 정리
        - html애서 요소란
          -요소 = 시작태그 + 콘텐츠 + 종료태그
            ```
              <p>콘텐츠</p>
            ```

        - 요소 특징
          - 1순위 : id값
            - #아이디값 -> 대기업, 글로벌기업 잘 사용
          - 2순위 : class값
            - 클레스값
          - 3순위 : 부모자식 관계
            - 직계
              - 부모 > 자식
            - 후손
              - 부모 손자
          - 4순위 : 의서결정샐렉터 or 속성셀렉터
          - 5순위 : 조합, 형제간 순서, 테그명 처리 등

In [40]:
# css selector로 탐색

# 매매 기준율 -> .sale
# 찾기
# 문서안에 일치되는 요소를 모두 다 찾는다 -> 복수형 -> 리스트
len(soup.select('.sale'))

for exchange in soup.select('.sale'):
 # print(exchange, exchang.text)
 print(exchange.text)
 #  break

1,483.90
1,632.07
1,021.79
200.60
191.04
44.85
1,899.61
3,854.39
1,040.82
1,759.74
148.22
882.55
818.30
64.81
1.48
39.05
0.42
392.61
218.59
135.58
395.30
4,815.98
3,936.60
404.00
2,092.95
28.94
42.47
1,097.52
329.50
8.80
407.11
2.86
1,097.52
17.20
5.29
12.21
25.77
71.08
246.69
5.71
75.00
17.28
4.00
382.22
4.95
11.11
11.46
0.33
0.55
10.75
327.91
266.82
185.35
0.71
11.22
0.11
0.37
623.68


In [41]:
# 통화명+코드명
for exchange in soup.select('.tit'):
 # print(exchange, exchang.text)
 print(exchange.text.strip())
 #  break

미국 USD
유럽연합 EUR
일본 JPY (100엔)
중국 CNY
홍콩 HKD
대만 TWD
영국 GBP
오만 OMR
캐나다 CAD
스위스 CHF
스웨덴 SEK
호주 AUD
뉴질랜드 NZD
체코 CZK
칠레 CLP
튀르키예 TRY
몽골 MNT
이스라엘 ILS
덴마크 DKK
노르웨이 NOK
사우디아라비아 SAR
쿠웨이트 KWD
바레인 BHD
아랍에미리트 AED
요르단 JOD
이집트 EGP
태국 THB
싱가포르 SGD
말레이시아 MYR
인도네시아 IDR 100
카타르 QAR
카자흐스탄 KZT
브루나이 BND
인도 INR
파키스탄 PKR
방글라데시 BDT
필리핀 PHP
멕시코 MXN
브라질 BRL
베트남 VND 100
남아프리카 공화국 ZAR
러시아 RUB
헝가리 HUF
폴란드 PLN
스리랑카 LKR
알제리 DZD
케냐 KES
콜롬비아 COP
탄자니아 TZS
네팔 NPR
루마니아 RON
리비아 LYD
마카오 MOP
미얀마 MMK
에티오피아 ETB
우즈베키스탄 UZS
캄보디아 KHR
피지 FJD


- 최종 결과물

```
  [
    {
      'name':'미국 USD',    # 통화코드명
      'std_rate':'1472.40,  # 매매 기준율
      'cash_buy':1490.      # 현금으로 살때
      'rate':1,0000         # 미환산기준율
      'code':'USD'          # 통화코드
    },
    {

    }
  ]
```

In [42]:
# 1. 해당 페이지에서 통화별로 일단 데이터 추출한다
#    통화의 총 개수 58개
#    .tbl_exchange > tbody > tr
#    클레스 값이 tbl_exchange인 요소의 직계자식들중  tbody의 직계 자식들중 tr을 모두 찾아라 => 58개
len(soup.select('.tbl_exchange > tbody > tr'))

58

In [43]:
# 국가별 통화 정보를 하나씩 추출 -> 범위가 좁혀짐
for na_exchange in soup.select('.tbl_exchange > tbody > tr'):
  # 통화명 추출
  print(na_exchange.select_one('.tit').text.strip())
  # 매매 기준율
  print(na_exchange.select_one('.sale').text.strip())
  # 현금으로 살때 -> td들 중에서 세번째 데이터임
  print(na_exchange.select_one('td:nth-of-type(3)').text.strip())
  break

미국 USD
1,483.90
1,509.86


In [44]:
# 현금을 사실때 => 모든 개별 환율정보의 3번째 데이터
# td:nth-child(3)

4월 9일

- 환율 정보를 가진 테이블을 특정하는 SELECTOR
  ```
    # 클레스값이 tbl_exchange인 요소를 찾아서 (.tbl_exchange)
    # 그 하위에 직계 자식(바로 밑에 있는 요서) (>)
    # 직계 자식들중에 tbody (요소명)을 특정
    # 그 하위에 직계 자식 (>)
    # 직계 자식들 중에서 tr들 (58개 존재함) (tr)
    # 그 tr들 중에서 5번째, 1번째, 2번째, ... 순번표기 (nth-child(5), 단 bs4에서는 nth-of-type(5)으로 변경 표기)
    
    .tbl_exchange > tbody > tr:nth-child(5)
  ```

In [55]:
# 모든 문서 -> soup 구성되어 있음
soup.select() # 모두 다 찾아라 => list()
soup.select_one() # 한개만 찾아라 => 단수
# .tbl_exchange > tbody > tr:nth-child(5)

TypeError: Tag.select() missing 1 required positional argument: 'selector'

In [ ]:
# 1. 현재 문서상에 존재하는 모든 환율관련 데이터를 <tr> 단위로 묶는다
# 환율 테이블에 tbody밑에 존재하는 모든 tr을 다 가져온다
soup.select('.tbi_sechange > tbody > tr')
len(trs) # 58개

In [ ]:
# 2. 모든 국가 통화 정보를 접근하여 개별 정보 추출 => 반복해서 수행
for tr in trs:
  # tr 정보 출력 -> 각국 통화정보가 있는 HTML처럼 보이는 객체 =>dom
  print(tr, type(tr))
  # 각국 통화정보에서 나머지 정보 추출 -> 전체 문서x,  현재 국가관련 파트만 체크
  # (통화명, 매매기준율, 현금으로 살때, 미화환산율)
  # 통화명,
  print( tr.select_one('.tit').text.strip() )
  # 매매기준율,
  print( tr.select_one('.sale').text.strip() )
  # 현금으로살때,
  print( tr.select_one('td:nth-child(3)').text.strip() )
  # 미화환산율
  print( tr.select_one('td:nth-child(7)').text.strip() )
  break #1회만 수행

In [ ]:
# html => ... => [{},{},..]
# for tr in trs:
#   print( tr.select_one('.tit').text.strip() )
#   print( tr.select_one('.sale').text.strip()
#   print( tr.select_one('td:nth-child(3)').text.strip() )
#   print( tr.select_one('td:nth-child(7)').text.strip() )

# 1. 결과에 맞춰서 틀을 만든다
# 2. 결과에 맞춰서 틀에 데이터를 세팅한다
results = [
  {
    # 각 국가별 통화 관련 정보 세팅
    # 통화명
    'name'     : tr.select_one('.tit').text.strip(),  # 통화명
    'std_rate' : tr.select_one('.sale').text.strip(), # 매매기준율
    'cash_buy' : tr.select_one('td:nth-child(3)').text.strip(), # 현챃구매시
    'd_rate'   : tr.select_one('td:nth-child(7)').text.strip(), # 미화 환산율
    # 메타 정보 (날짜, 공급기관, 회차)
  }
  for tr in trs
]
print(results)

## 환율의 메타 정보 추출

- 대상
  - 2025.04.09 10:06 하나은행 기준 고시회차 164회
  - 년월일시분초, 은행, 회차
  - 타겟 사이트
    - https://finance.naver.com/marketindex/
  - css selector
    - 년월일시분포
      ```
        .exchange_info > .date
        or
        여기서는 아래 표현도 가능함
        .date
      ```
    - 은행
      ```
        .exchange_info > .standard
        or
        .standard
      ```
    - 회차
      ```
        .exchange_info > .round > em
        or
        .round > em
      ```

In [52]:
# part 1 -> 요청 -> 응담 -> dom 구성 : soup 생성
res = req.urlopen('https://finance.naver.com/marketindex/')
soup = BeautifulSoup(res, 'html5lib')
# part 2 -> 년월일시분초, 은행, 회차 추출
meta_data = [
  soup.select_one('.date').text.strip(),
  soup.select_one('.standard').text.strip(),
  soup.select_one('.round>em').text.strip(),
]
# 확인
meta_data

['2025.04.09 10:22', '하나은행 기준', '207']

# 최종

In [ ]:
results = [
  {
    # 각 국가별 통화 관련 정보 세팅
    # 통화명
    'name'     : tr.select_one('.tit').text.strip(),  # 통화명
    'std_rate' : tr.select_one('.sale').text.strip(), # 매매기준율
    'cash_buy' : tr.select_one('td:nth-child(3)').text.strip(), # 현챃구매시
    'd_rate'   : tr.select_one('td:nth-child(7)').text.strip(), # 미화 환산율
    # 메타 정보 (날짜, 공급기관, 회차)
    'date'     : meta_data[0],
    'standard' : meta_data[1],
    'round'    : meta_data[2]
  }
  for tr in trs
]
print(results)

# 나머지 처리

- 위에서 만들어진 데이터 -> DataFrame -> DB입력
- 하루에 1회차 ~ 최종회차 주기적으로 반복 자동실행 => 자동화 필요